In [97]:
# Imports GEE and Initializes with Cloud Project
import ee
import geemap

ee.Initialize(project="ee-nathanielpaulrobinson")
print(ee.__version__)

1.1.4


In [98]:
# Load Data to Use in Analyses

# Load Private Assets
asset_path = "projects/ee-nathanielpaulrobinson/assets/NTRI-data/"
ntri_villages = ee.FeatureCollection(asset_path + "NTRI-villages")
ntri_extent = ee.FeatureCollection(asset_path + "NTRI-boundary")
glad_bareground = ee.Image(asset_path + "bare_00N_030E")

# Load Public Assets
mod44b = ee.ImageCollection("MODIS/006/MOD44B")
rap_veg = ee.ImageCollection("projects/rap-data-365417/assets/vegetation-cover-v3")
hansen_gfc = ee.Image("UMD/hansen/global_forest_change_2023_v1_11")
protected_areas = ee.FeatureCollection("WCMC/WDPA/current/polygons")

# Load Geometry - for RAP comparison
geometry = ee.Geometry.Polygon(
    [
        [
            [-113.07017637550278, 42.88855499739364],
            [-113.07017637550278, 32.857108964921466],
            [-97.33775450050278, 32.857108964921466],
            [-97.33775450050278, 42.88855499739364],
        ]
    ],
    None,
    False,
)

In [99]:
# Key Variables and Constants

# Export Folder
export_folder = "ntri_bg_data"

# Projection and Scale Constants
mod44_projection = mod44b.first().projection()
mod44_scale = mod44_projection.nominalScale().getInfo()
mod44_crs = mod44_projection.crs().getInfo()
rap_projection = rap_veg.first().projection()
glad_projection = glad_bareground.projection()
glad_scale = glad_projection.nominalScale().getInfo()
glad_crs = glad_projection.crs().getInfo()

In [100]:
# Village Selection
assessment_village_names = [
    "Eleng'ata Dapash",
    "Engaruka chini",
    "Kakoi",
    "Katikati",
    "Kimana",
    "Kitwai A",
    "Losirwa",
    "Matale A",
    "Nadonjukin",
    "Ngoswak",
    "Olchoroonyokie",
    "Sangaiwe",
]

# Filter Villag Collection to Assessment Villages
assessment_villages = ntri_villages.filter(
    ee.Filter.inList("Village", assessment_village_names)
)

In [101]:
# Create Pixel Area Image for Area Analyses
land_mask = hansen_gfc.select("datamask").eq(1)
area = ee.Image.pixelArea().multiply(0.0001).updateMask(land_mask).rename("area")

In [102]:
# MOD44 - RAP comparison

# MOD44B Processing
nonvegetated_band = "Percent_NonVegetated"
pre_start_date = "2010"
pre_end_date = "2015"
post_start_date = "2016"
post_end_date = "2021"

mod44b_nonvegetated = mod44b.select(nonvegetated_band)

pre_nonvegetated = (
    mod44b_nonvegetated.filterDate(pre_start_date, pre_end_date)
    .mean()
    .rename("pre_nonvegetated_mod44")
)
post_nonvegetated = (
    mod44b_nonvegetated.filterDate(post_start_date, post_end_date)
    .mean()
    .rename("post_nonvegetated_mod44")
)

mod44_bgr = pre_nonvegetated.addBands(post_nonvegetated)


# RAP Processing
rap_bare_ground_band = "BGR"
rap_bgr_collection = rap_veg.select(rap_bare_ground_band)
rap_projection = rap_veg.first().projection()

pre_rap_bgr = (
    rap_bgr_collection.filterDate(pre_start_date, pre_end_date)
    .mean()
    .rename("pre_nonvegetated_rap")
)


post_rap_bgr = (
    rap_bgr_collection.filterDate(post_start_date, post_end_date)
    .mean()
    .rename("post_nonvegetated_rap")
)

rap_bgr = pre_rap_bgr.addBands(post_rap_bgr).setDefaultProjection(rap_projection)

# RAP Processing
rap_bare_ground_band = "BGR"
rap_bgr_collection = rap_veg.select(rap_bare_ground_band)


pre_rap_bgr = (
    rap_bgr_collection.filterDate(pre_start_date, pre_end_date)
    .mean()
    .rename("pre_nonvegetated_rap")
)


post_rap_bgr = (
    rap_bgr_collection.filterDate(post_start_date, post_end_date)
    .mean()
    .rename("post_nonvegetated_rap")
)

rap_bgr = pre_rap_bgr.addBands(post_rap_bgr).setDefaultProjection(rap_projection)

# Combine MOD44 and RAP Datasets
rap_bgr_mod = rap_bgr.reproject(mod44_projection)
nonvegetated_rap_mod44 = rap_bgr_mod.addBands(mod44_bgr)

# Create Set of Random Points to Sample Datasets
random_points = ee.FeatureCollection.randomPoints(geometry, 10000)
bare_stats = nonvegetated_rap_mod44.reduceRegions(
    collection=random_points,
    scale=mod44_scale,
    crs=mod44_crs,
    reducer=ee.Reducer.mean(),
)

# Set up Export Task
mod_rap_export = ee.batch.Export.table.toDrive(
    collection=bare_stats,
    description="mod44_rap_stats",
    fileFormat="CSV",
    fileNamePrefix="mod44_rap_stats",
    folder=export_folder,
)

In [103]:
# Combine GLAD BGR and MOD44
mod_bgr_2010 = (
    mod44b_nonvegetated.filterDate("2010", "2011")
    .first()
    .divide(100)
    .multiply(area)
    .rename("mod_bgr")
    .addBands(area)
)
glad_bgr = glad_bareground.divide(100).multiply(area).rename("glad_bgr").addBands(area)

# Village area stats created for MOD44 dataset at MOD44 resolution and crs
village_mod_stats = mod_bgr_2010.reduceRegions(
    collection=assessment_villages,
    scale=mod44_scale,
    crs=mod44_crs,
    reducer=ee.Reducer.sum(),
).select(["Vil_Mtaa_N", "area", "mod_bgr"])

# Village area stats created for GLAD dataset at GLAD resolution and crs
village_glad_stats = glad_bgr.reduceRegions(
    collection=assessment_villages,
    scale=glad_scale,
    crs=glad_crs,
    reducer=ee.Reducer.sum(),
).select(["Vil_Mtaa_N", "area", "glad_bgr"])

# Join output tables on village area match (village names have a repeat which may cause join error)
max_diff_filter = ee.Filter.maxDifference(
    difference=8, leftField="area", rightField="area"
)

save_best_join = ee.Join.saveBest(matchKey="best_village", measureKey="areaDiff")

join = save_best_join.apply(village_mod_stats, village_glad_stats, max_diff_filter)


# Clean Output Tables
def clean_join(feature):
    best_feature = ee.Feature(feature.get("best_village"))
    bg_stat = best_feature.get("glad_bgr")
    return feature.set({"glad_bgr": bg_stat})


joined_tables = join.map(clean_join).select(
    ["Vil_Mtaa_N", "area", "glad_bgr", "mod_bgr"]
)

mod_glad_export = ee.batch.Export.table.toDrive(
    collection=joined_tables,
    description="village_bare_stats",
    fileFormat="CSV",
    fileNamePrefix="village_bare_stats",
    folder=export_folder,
)

In [104]:
# Run up Export Tasks
mod_rap_export.start()
mod_glad_export.start()